In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()

In [ ]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code,load_node
from tools.aiida_support import wait_for_node_finished

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [ ]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home,'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_opt', 
                          computer_local, 
                          '/home/max/Documents/alamode/bin/alm')
code_alm_opt = code
code = get_or_create_code('alamode.anphon', 
                          computer_local, 
                          '/home/max/Documents/alamode/bin/anphon')
code_anphon = code
code = get_or_create_code('alamode.analyze_phonons', 
                          computer_local, 
                          '/home/max/Documents/alamode/bin/analyze_phonons')
code_analyze_phonons = code
code_alm_opt, code_anphon, code_analyze_phonons

In [ ]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path

In [ ]:
from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)

In [ ]:
g_prefix = nodebank.load('cubic_prefix')

g_norder = nodebank.load(f"{g_prefix.value}_norder")
g_norder, g_prefix

In [ ]:
g_cwd = nodebank.load(f"{g_prefix.value}_cwd") # 保存directory
# g_dispfile_folder = nodebank.load("cubic_dispfile_folder")
# g_displace_result = nodebank.load("cubic_displace_result")
g_superstructure = nodebank.load("superstructure") # 長周期構造

g_cwd

In [ ]:
node = nodebank.load(f'{g_prefix.value}_extract')
g_DFSET = node.outputs.dfset
g_DFSET.attributes

# alm opt 

In [ ]:
g_norder

In [ ]:
g_diag = nodebank.load("factor")
g_diag.get_array('factor')

In [ ]:
g_primstructure = nodebank.load("primstructure")
g_primstructure

In [ ]:

g_almprefix = "_".join([g_primstructure.get_formula(),"x".join(map(str,g_diag.get_array("factor").reshape(-1))), "cubic"])
g_almprefix = Str(g_almprefix)

g_wd = nodebank.load_or_dump("cubic_almprefix",g_almprefix) # subdirectory名
g_DFSET, g_almprefix, g_wd

In [ ]:
g_fc2xml = nodebank.load("anphonfile_xml") # harmonic force constant
g_fc2xml

In [ ]:
#g_alm_opt = run(builder)

In [ ]:
#codename = "alm_opt@tutor"
#code = Code.get_from_string(codename)
code = code_alm_opt

builder = code.get_builder()
builder.structure = g_superstructure
builder.prefix = g_prefix
builder.cwd = Str(os.path.join(g_cwd.value,f"{g_prefix.value}_alm_opt")) # 保存directory
builder.norder = g_norder
builder.dfset = g_DFSET # displacementとforce
builder.fc2xml =  g_fc2xml # force constant of the harmonic term

key = f"{g_prefix.value}_alm_opt"

g_alm_opt = nodebank.load_code_or_wait_for_node(key, builder, sec=10)


In [ ]:
g_alm_opt.outputs.results.attributes

In [ ]:
g_alm_opt.outputs.input_ANPHON

## thermal conductivity


In [ ]:
g_primstructure = nodebank.load("primstructure")
g_standardizedstructure = nodebank.load("standardizedstructure")

g_diag = nodebank.load("factor")
g_qmesh = List(list=[5,5,5])

g_almprefix = "_".join([g_standardizedstructure.get_formula(),
                        "x".join(map(str,g_diag.get_array("factor").reshape(-1))),
                        g_prefix.value,
                       "q"+"x".join(map(str,g_qmesh.get_list()))
                       ])
print(g_almprefix)

g_wd_dos = Str(g_almprefix)
g_wd_dos, g_cwd

In [ ]:
g_property = Str(os.path.join(g_cwd.value, f'{g_prefix.value}_property'))
g_property

In [ ]:
#g_RTAfuture = run(builder)

In [ ]:
#codename = "anphon@tutor"
#code = Code.get_from_string(codename)
code = code_anphon

builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_wd
builder.cwd = g_property
builder.norder = g_norder
builder.fcsxml = g_alm_opt.outputs.input_ANPHON
builder.mode = Str("RTA")
builder.qmesh = g_qmesh

key = "RTA"

g_RTAfuture = nodebank.load_code_or_wait_for_node(key, builder, 10)


In [ ]:
g_RTAfuture

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
from alamode_aiida.io import load_anphon_kl, load_anphon_kl_spec

In [ ]:
g_RTAfuture.outputs.kl_file

In [ ]:
g_df = load_anphon_kl(g_RTAfuture.outputs.kl_file)
g_df.plot(x="Temperature [K]", y="Thermal Conductivity_xx [W/mK]")
import matplotlib.pyplot as plt
plt.xscale("log")

In [ ]:
g_RTAfuture.outputs.result_file

In [ ]:
#codename = "analyze_phonon@tutor"
#code = Code.get_from_string(codename)
code = code_analyze_phonons

builder = code.get_builder()
builder.cwd = g_property
builder.prefix = g_wd
builder.calc = Str("kappa_boundary")
builder.file_result = g_RTAfuture.outputs.result_file
builder.param = Dict(dict={"size": 1e+6})

key = "ap_kappa"

g_ap_kappa = nodebank.load_code_or_wait_for_node(key, builder, 10)


In [ ]:
g_ap_kappa.outputs.kappa_boundary_file

In [ ]:
g_RTAfuture.outputs.result_file.attributes
    

In [ ]:
code = code_analyze_phonons

builder = code.get_builder()
builder.cwd = g_property
builder.prefix = g_wd
builder.calc = Str("tau")
# builder.size = Float(1.0e+6)
builder.file_result = g_RTAfuture.outputs.result_file
builder.param = Dict(dict={"temp": 300.0})

key = "ap_tau"

g_ap_tau = nodebank.load_code_or_wait_for_node(key, builder, 10)


In [ ]:
code = code_analyze_phonons

builder = code.get_builder()
builder.cwd = g_property
builder.prefix = g_wd
builder.calc = Str("cumulative")
# builder.size = Float(1.0e+6)
builder.file_result = g_RTAfuture.outputs.result_file
builder.param = Dict(dict={"temp": 300.0, "length": "10000:5"})

key = "ap_cum"
g_ap_cum = nodebank.load_code_or_wait_for_node(key, builder, 10)


In [ ]:
values = g_ap_cum.outputs.cumulative.get_array("values")
values

In [ ]:
values = g_ap_cum.outputs.cumulative.get_array("values").astype(float)
splitted_header = g_ap_cum.outputs.cumulative.get_array("columns")
df = pd.DataFrame(values, columns=splitted_header)
df

In [ ]:
df.plot(x= "L [nm]", y='kappa xx [W/mK]')

## anphon spectra

In [ ]:
#codename = "anphon@tutor"
#code = Code.get_from_string(codename)
code = code_anphon

builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_wd
builder.cwd = g_property
builder.norder = g_norder
builder.fcsxml = g_alm_opt.outputs.input_ANPHON
builder.mode = Str("RTA")
builder.kappa_spec = Int(1) # spectra
builder.qmesh = g_qmesh
eparam = {"EMIN":0, "EMAX":550, "DELTA_E": 10.0}
builder.param= Dict(dict={"general": eparam})

key = "RTAspec"
g_RTAspec = nodebank.load_code_or_wait_for_node(key, builder, 10)


In [ ]:
g_df = load_anphon_kl_spec(g_RTAspec.outputs.kl_spec_file)

In [ ]:
print(np.unique(g_df["Temperature [K]"].values))
g_df_select = g_df[g_df["Temperature [K]"]==300]

In [ ]:
g_df_select.plot(x=g_df.columns[1], y=g_df.columns[2])